### IMPORT DEPENDENCIES

In [3]:
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu117

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu117


In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
import re
import pandas as pd
import numpy as np

### INSTANTIATE  MODEL

In [6]:
tokenizer = AutoTokenizer.from_pretrained('cardiffnlp/twitter-roberta-base-sentiment-latest')
model = AutoModelForSequenceClassification.from_pretrained('cardiffnlp/twitter-roberta-base-sentiment-latest')

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### ENCODE AND CALCULATE SENTIMENT

In [7]:
tokens = tokenizer.encode('Ok. I admit it. Iâ€™m starting to use my @coinbase card more than my @Apple card now. I like the 4% #crypto back no matter how I use the card. Just some foreshadowing of how much abuse the @investvoyager card will get when I get it. #VGX', return_tensors='pt')

In [9]:
result = model(tokens)

In [12]:
result.logits

tensor([[-0.9049,  0.3853,  0.2639]], grad_fn=<AddmmBackward0>)

In [13]:
int(torch.argmax(result.logits))+1

2

### COLLECT DATA FROM CSV

In [17]:
df = pd.read_csv('C:\\Users\\lucy22\\Sentiment Analysis\\Cryptocurrency_data.csv ')

In [18]:
df

,Unnamed: 0,Date,User,Tweet
0,0,2023-01-26 12:00:02+00:00,CoinexF,💡Découvrez le Top 5 des #cryptomonnaies les pl...
1,1,2023-01-26 04:50:54+00:00,ri_meyers,The opportunity is still here to grab some low...
2,2,2023-01-25 18:53:48+00:00,___INFINITY___,"🔥Join and win for $25,000 rewards for @coinbas..."
3,3,2023-01-25 18:12:35+00:00,Jay_Brockett,Day 140 kindly asking @coinbase to List #VOLTI...
4,4,2023-01-25 17:11:30+00:00,BennyAtTheBank,Look how helpful @MetaMask is with this email ...
...,...,...,...,...
4262,4262,2019-12-17 16:20:38+00:00,veradittakit,Pantera is proud to be leading the Series A ro...
4263,4263,2019-12-15 19:49:03+00:00,Concept211,I can't recommend @ChicoCrypto's research enou...
4264,4264,2019-12-13 14:40:36+00:00,Brad_Laurie,If you are a #crypto solider with a 'only-one-...
4265,4265,2019-12-12 17:50:50+00:00,TheCryptoZombie,What’s the best price to buy #Bitcoin at? In t...


### LOAD REVIEWS INTO DATAFRAME AND SCORE
#### Data Score:-
Here Sentiment Score of 1 denotes negative sentiment, 2 is for Neutral sentiment and 3 is for Positive Sentiment.

In [20]:
df['Tweet'].iloc[0]

'💡Découvrez le Top 5 des #cryptomonnaies les plus recherchées sur CoinEx le 19/01/2023\n\n1️⃣ $APT @Aptos_Network\n2️⃣ $DYDX @dYdX\n3️⃣ #BABYDOGE @BabyDogeCoin\n4️⃣ $BTC @Bitcoin\n5️⃣ $BONK @bonk_inu\n\nSource 👉 https://t.co/3bKMrxHxID\n\n#CoinEx #cryptosphère #Crypto #CryptoTrading https://t.co/KRnAUVTh2t'

In [21]:
def sentiment_score(Tweet):
    tokens = tokenizer.encode(Tweet, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1
    

In [22]:
sentiment_score(df['Tweet'].iloc[0])

2

In [23]:
df['sentiment'] = df['Tweet'].apply(lambda x: sentiment_score(x[:512]))

In [24]:
df

,Unnamed: 0,Date,User,Tweet,sentiment
0,0,2023-01-26 12:00:02+00:00,CoinexF,💡Découvrez le Top 5 des #cryptomonnaies les pl...,2
1,1,2023-01-26 04:50:54+00:00,ri_meyers,The opportunity is still here to grab some low...,3
2,2,2023-01-25 18:53:48+00:00,___INFINITY___,"🔥Join and win for $25,000 rewards for @coinbas...",3
3,3,2023-01-25 18:12:35+00:00,Jay_Brockett,Day 140 kindly asking @coinbase to List #VOLTI...,2
4,4,2023-01-25 17:11:30+00:00,BennyAtTheBank,Look how helpful @MetaMask is with this email ...,3
...,...,...,...,...,...
4262,4262,2019-12-17 16:20:38+00:00,veradittakit,Pantera is proud to be leading the Series A ro...,3
4263,4263,2019-12-15 19:49:03+00:00,Concept211,I can't recommend @ChicoCrypto's research enou...,3
4264,4264,2019-12-13 14:40:36+00:00,Brad_Laurie,If you are a #crypto solider with a 'only-one-...,2
4265,4265,2019-12-12 17:50:50+00:00,TheCryptoZombie,What’s the best price to buy #Bitcoin at? In t...,2


### CONVERTING DATAFRAME TO CSV :-

In [25]:
df.to_csv("sentiment.csv")